In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
dataframe = pd.read_csv("./Dataset/Balloon Race_ Data Breaches - LATEST - breaches.csv")

In [3]:
dataframe_raw = dataframe[1:]

In [4]:
dataframe_raw2 = dataframe_raw.drop(['alternative name', 'story', 'interesting story', 'displayed records', 'Unnamed: 11', '1st source link', '2nd source link'], axis = 1)
dataframe_raw2

,organisation,records lost,year,date,sector,method,data sensitivity,source name,ID
1,Plex,"15,000,000",2022,Aug 2022,web,hacked,1,Ars technica,418.0
2,Twitter,"5,400,000",2021,Dec 2021,web,hacked,2,Bleeping Computer,419.0
3,Shanghai Police,"500,000,000",2022,Jul 2022,financial,hacked,5,The Register,420.0
4,"City of Amagasaki, Japan","500,000",2022,Jun 2022,government,oops!,3,BBC,421.0
5,Dubai Real Estate Leak,"800,000",2022,May 2022,financial,inside job,1,E24,417.0
...,...,...,...,...,...,...,...,...,...
413,Hewlett Packard,"200,000",2006,Mar 2006,"tech, retail",lost device,2,Computer Weekly,6.0
414,Ameritrade Inc.,"200,000",2005,Apr 2005,finance,lost device,2,NBC,5.0
415,Citigroup,"3,900,000",2005,Jun 2005,finance,lost device,3,NY Times,4.0
416,Cardsystems Solutions Inc.,"40,000,000",2005,Jun 2005,finance,hacked,3,Wired,3.0


In [5]:
# trim sector
dataframe_raw2['sector'] = dataframe_raw2['sector'].str.strip()

# trim method
dataframe_raw2['method'] = dataframe_raw2['method'].str.strip()

# trim method
dataframe_raw2 = dataframe_raw2.rename(columns = lambda x : x.strip())
dataframe_raw2.columns = dataframe_raw2.columns.str.replace(' ', '_')

# select Monthname from date
dataframe_raw2['month'] = pd.to_datetime(dataframe_raw2.date).dt.to_period('m').dt.strftime('%B')

# transfrom records lost to int
dataframe_raw2['records_lost'] = dataframe_raw2['records_lost'].str.replace(",", "").astype(int)

# remove NaN values
dataframe_raw2['data_sensitivity'] = dataframe_raw2['data_sensitivity'].fillna(0) 

# transform string columns into int
dataframe_raw2[['ID', 'year', 'data_sensitivity']] = dataframe_raw2[['ID', 'year', 'data_sensitivity']].astype(int)


# split sector into sector 1 and sector 2
dataframe_raw2[['sector_1', 'sector_2']] = dataframe_raw2['sector'].str.split(',', n = 1, expand = True)
dataframe_raw2['sector_2'] = dataframe_raw2['sector_2'].fillna('No other sector') 

# add Sensitivity describtion
condlist = [
                (dataframe_raw2['data_sensitivity'] == 0),
                (dataframe_raw2['data_sensitivity'] == 1),
                (dataframe_raw2['data_sensitivity'] == 2),
                (dataframe_raw2['data_sensitivity'] == 3),
                (dataframe_raw2['data_sensitivity'] == 4),
                (dataframe_raw2['data_sensitivity'] == 5)
            ]

choicelist = ['Unknown', 'Just email address/Online information', 'SSN/Personal details', 'Credit card information', 'Health & other personal records', 'Full details']

dataframe_raw2['data_sensitivity_text'] = np.select(condlist, choicelist)

In [6]:
dataframe_raw2

,organisation,records_lost,year,date,sector,method,data_sensitivity,source_name,ID,month,sector_1,sector_2,data_sensitivity_text
1,Plex,15000000,2022,Aug 2022,web,hacked,1,Ars technica,418,August,web,No other sector,Just email address/Online information
2,Twitter,5400000,2021,Dec 2021,web,hacked,2,Bleeping Computer,419,December,web,No other sector,SSN/Personal details
3,Shanghai Police,500000000,2022,Jul 2022,financial,hacked,5,The Register,420,July,financial,No other sector,Full details
4,"City of Amagasaki, Japan",500000,2022,Jun 2022,government,oops!,3,BBC,421,June,government,No other sector,Credit card information
5,Dubai Real Estate Leak,800000,2022,May 2022,financial,inside job,1,E24,417,May,financial,No other sector,Just email address/Online information
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,Hewlett Packard,200000,2006,Mar 2006,"tech, retail",lost device,2,Computer Weekly,6,March,tech,retail,SSN/Personal details
414,Ameritrade Inc.,200000,2005,Apr 2005,finance,lost device,2,NBC,5,April,finance,No other sector,SSN/Personal details
415,Citigroup,3900000,2005,Jun 2005,finance,lost device,3,NY Times,4,June,finance,No other sector,Credit card information
416,Cardsystems Solutions Inc.,40000000,2005,Jun 2005,finance,hacked,3,Wired,3,June,finance,No other sector,Credit card information


In [7]:
dataframe_raw2.dtypes

organisation             object
records_lost              int32
year                      int32
date                     object
sector                   object
method                   object
data_sensitivity          int32
source_name              object
ID                        int32
month                    object
sector_1                 object
sector_2                 object
data_sensitivity_text    object
dtype: object

In [8]:
dataframe_raw2 = dataframe_raw2[['ID', 'organisation', 'records_lost', 'date', 'year', 'month', 'sector', 'sector_1', 'sector_2', 
                                 'method', 'data_sensitivity', 'data_sensitivity_text', 'source_name']]
dataframe_raw2

,ID,organisation,records_lost,date,year,month,sector,sector_1,sector_2,method,data_sensitivity,data_sensitivity_text,source_name
1,418,Plex,15000000,Aug 2022,2022,August,web,web,No other sector,hacked,1,Just email address/Online information,Ars technica
2,419,Twitter,5400000,Dec 2021,2021,December,web,web,No other sector,hacked,2,SSN/Personal details,Bleeping Computer
3,420,Shanghai Police,500000000,Jul 2022,2022,July,financial,financial,No other sector,hacked,5,Full details,The Register
4,421,"City of Amagasaki, Japan",500000,Jun 2022,2022,June,government,government,No other sector,oops!,3,Credit card information,BBC
5,417,Dubai Real Estate Leak,800000,May 2022,2022,May,financial,financial,No other sector,inside job,1,Just email address/Online information,E24
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,6,Hewlett Packard,200000,Mar 2006,2006,March,"tech, retail",tech,retail,lost device,2,SSN/Personal details,Computer Weekly
414,5,Ameritrade Inc.,200000,Apr 2005,2005,April,finance,finance,No other sector,lost device,2,SSN/Personal details,NBC
415,4,Citigroup,3900000,Jun 2005,2005,June,finance,finance,No other sector,lost device,3,Credit card information,NY Times
416,3,Cardsystems Solutions Inc.,40000000,Jun 2005,2005,June,finance,finance,No other sector,hacked,3,Credit card information,Wired


In [9]:
# dataframe_raw2.sector.unique()

## Save Dataset to Folder

In [10]:
dataframe_raw2.to_csv("./Dataset/Dataset.csv", index = False)